# 实战_MCP工具继承智能体

## 简介
课程作业旨在让学员熟悉MCP工具如何使用，以及了解大模型调用工具实现多轮对话的底层原理，以解决特定的任务或实现特定的功能。
## 学习目标
- 能够开发并以sse方式部署MCP工具并调用，能够集成多个mcp工具配合解决复杂问题。
## 作业安排
- 项目目标：开发部署两个MCP工具：数据库处理工具，ocr图像识别工具，并集成这两个工具和大模型服务实现一个对话智能体实现多轮调用工具解决问题。
  
- 要求：<br>
  本地部署mysql服务（或者使用sqlite，建议尝试mysql，sqlite功能和量级有限一般正式开发不会用这个）；<br>
  选择一个大模型，封装一个模型调用模块进行调用；<br>
  封装两个MCP工具：数据库处理工具，ocr图像识别工具，以sse方式部署到本地不同的端口；<br>
  集成两个工具和模型调用模块，实现工具调用并存储对应的历史纪录，用哪个工具由模型自己决定；<br>
  要求至少能解决的问题：本地预先存放一张图片，在多次对话中要求模型实现：使用ocr工具识别图片内容并以图片中数据为基础设计数据库表并存入本地数据库；<br>
- 拓展内容（选作）：<br>
  实现单次对话多轮工具调用，即发布任务后，模型自己设计任务，并且一次性执行多次不同的工具组合调用<br>
  实现一个前端，可以使用gradio实现，前端内容包括基础对话功能，图片上传功能（上传后能被模型识别到）<br>
  显示工具调用功能<br>
  集成mcp服务时，下放给的代码示例为单次连接的示例，出了async with的代码块session会失效，学员可以尝试改编代码手动控制多线程上下文管理的实现，从而能够复用session<br>
  

  
- 提交内容：<br>
  项目报告（包括模型选择、实验设计、结果分析等）。<br>
  应用代码或应用Demo视频（二选一）：应用Demo需要完整的展示应用功能。<br>



## 需要注意的事项以及操作步骤
注意：<br>
mysql部署后需要启动，代码中链接需要mysql-connector-python库，使用的python版本建议使用3.10，代码中连接时提供host，port，账密，数据库信息。<br>
ocr可以使用阿里的vl模型，具体调用方法可以参照官方文档。<br>
mcp服务启动后不要关闭终端（除非后台运行，后台运行的话需要自己搜索启动和关闭命令），新开一个终端跑主体程序<br>

In [ ]:
#mysql工具
from mcp.server import FastMCP
# 初始化 FastMCP 服务器时，可以定义服务名和端口号
app = FastMCP('mysql-server',port=8001)

@app.tool()
async def execute_sql(query: str) -> str:
    """
    这里写工具的描述信息
    """
    #在这里定义具体的执行逻辑
    pass
#以sse方式启动
if __name__ == "__main__":
    app.run(transport='sse')

In [ ]:
#ocr工具
from mcp.server.fastmcp import FastMCP

# 初始化 FastMCP 服务器
mcp = FastMCP("ocr",port=8002)

@mcp.tool()
async def describe_image(image_path):
    """
    这里写工具的描述信息
    """
    #在这里定义具体的执行逻辑
    pass

if __name__ == "__main__":
    mcp.run(transport='sse')

In [ ]:
#mcp服务调用测试方法
import asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client
 
async def main():
    # 连接到SSE服务器，最好使用异步调用，这里填写mcp服务启动后的端口号
    async with sse_client(url="http://127.0.0.1:8000/sse") as streams:
        #建立链接，自己集成的时候，可以初始化的时候就链接好
        async with ClientSession(*streams) as session:
            # 初始化会话
            await session.initialize()
            
            # 列出可用工具
            tools_response = await session.list_tools()
            print("Available tools:")
            for tool in tools_response.tools:
                # print(tool)
                print(f" - {tool.name}:")
                print(f"   - 工具描述: {tool.description}")
            #这里根据自己的mcp服务的函数名和参数名进行更改
            response = await session.call_tool('describe_image', {'image_path': r'图片路径'})
            print(response)

if __name__ == "__main__":
    asyncio.run(main())

## 提交内容：
提交word或pdf文档，提交文件标题为MCP实战作业报告+学员姓名
文档内容需要包含以下几个部分：
1. mcp服务介绍
- 1.1 mcp服务对应功能细节
- 1.2 mcp服务具体实现
2. 模型选择以及集成过程
- 2.1 模型选择
- 2.2 如何实现多轮工具调用
3. 总结
- 3.1 应用功能展示（截图）
- 3.2 作业感想
- 3.3（可选）课程反馈

应用代码或应用Demo视频（二选一）：应用Demo需要完整的展示应用功能。

## 实战提交链接

https://candidate.aidexianzhaopinguan.com/exam/97ffe5e7837e48d5aa836d6831076cf9

- 点击作业提交链接，进入到AI面试系统，限时60分钟提交，作业中如需提交代码文件和报告等，请同学们提前做好作业压缩成.zip文件；
- .zip文件可以通过AI面试系统附件方式上传；
- 登录界面中需填入你的真实姓名和与登录小鹅通一致的手机号，方便老师统一批改；
- 提交好作业后，告诉一下你的班班老师，我们方便快速为你批改。